In [10]:
import pandas
import numpy

from sklearn import preprocessing
from sklearn import naive_bayes
from sklearn import tree
from sklearn import neighbors
from sklearn import model_selection
from sklearn.model_selection import cross_val_score

from numpy import var
from random import choices
from random import randint
import math
import copy

#INITIAL_P =  30
#NUM_GENERATIONS = 10
#NUM_MAX_AT = 8
#dicc_fitness = dict()
#data =  pandas.read_csv('conjuntos_de_datos/parkinson.csv', header=0)
#iter_equal_C = 5
#probMutate=0.6
#probReproduce=0.9
#umbral_var = 0.001
#fitness_pop = []  
#numRepe = 5

In [12]:
def randomGene():
    return randint(0,1)

def initialPop(columns,INITIAL_P): #Generates the initial population
    res=[]
    for i in range(0,INITIAL_P):
        res.append(initialChromosome(columns))
    return res

def initialChromosome(columns): #Generates the initial chromosomes
    res = []
    for i in range(0,columns-1):
        res.append(randomGene())
    return res

def fitness (c,NUM_MAX_AT,data,numRepe,fitness_pop,dicc_fitness):
    if (str(c) in dicc_fitness):
        f = dicc_fitness.get(str(c))
        fitness_pop.append(f)
        return f
    else:
        f = 0
        atributos = selectAtributes(c,data)
        objetivo = data.iloc[:,-1]
        clasif_CART = tree.DecisionTreeClassifier(
            max_depth=5,  # máxima profundidad del árbol
            random_state=54321)
        score = 0
        for i in range(0,numRepe):
            scores = cross_val_score(clasif_CART,atributos
                ,objetivo,cv=10,scoring='balanced_accuracy')
            score +=sum(scores)/len(scores)
        num_at = atributos.shape[1]
        f = 100*(score/numRepe)-15*penalization(num_at,NUM_MAX_AT)
        dicc_fitness[str(c)] = (f)
        fitness_pop.append(f)
        return f
def penalization(num_at,NUM_MAX_AT):
    if num_at > NUM_MAX_AT:
        return (num_at-NUM_MAX_AT)**2
    else:
        return 0



def selectAtributes(c,data):
    indexAttributes = []
    for i in range(0,len(c)):
        if(c[i]==1):
            indexAttributes.append(True)
        else:
            indexAttributes.append(False)
    indexAttributes.append(False)
    return data.iloc[:,indexAttributes]

def reproduce (c1, c2):
    
    newC1 = []
    newC2 = []
    half = math.floor(len(c1)/2)
    
    for i in range (0,len(c1)):
        if(i<half):
            newC1.append(c1[i])
            newC2.append(c2[i])
        else:
            newC1.append(c2[i])
            newC2.append(c1[i])
    
    newC1 = all0(newC1)
    newC2 = all0(newC2)
    return (newC1,newC2)


def all0(res):
    if 1 not in res:
        random = randint(0,len(res)-1)
        res[random] = 1
    return res

def mutate (c):
    
    ig = randint(0,len(c)-1)
    for i in range(0,len(c)):
        if(i == ig):
            if(c[i] == 0): 
                c[i] = 1 
            else: 
                c[i] = 0  
    c = all0(c)
    return c


def nextGeneration(pop,mutateProbability,reproduceProbability,INITIAL_P,NUM_MAX_AT,data,numRepe,fitness_pop,dicc_fitness):
    res = []
    half = math.floor(len(pop)/2)
    for i in range(0,half):
        cx = selection(pop,INITIAL_P,NUM_MAX_AT,data,numRepe,fitness_pop,dicc_fitness)
        cy = selection(pop,INITIAL_P,NUM_MAX_AT,data,numRepe,fitness_pop,dicc_fitness)
        if choices(population=[True, False],weights=reproduceProbability):
            newC = reproduce(cx, cy)
            if choices(population=[True, False],weights=mutateProbability):
                cx = mutate(newC[0])
                cy = mutate(newC[1])
        res.append(cx)
        res.append(cy)
    return res
def selection(pop,INITIAL_P,NUM_MAX_AT,data,numRepe,fitness_pop,dicc_fitness):
    selected = choices(population=pop,k = math.ceil(INITIAL_P/10))
    return bestSolution(selected,NUM_MAX_AT,data,numRepe,fitness_pop,dicc_fitness)
    

def bestSolution (pop,NUM_MAX_AT,data,numRepe,fitness_pop,dicc_fitness):
    
    bestC = pop[0]
    bestFitness = fitness(pop[0],NUM_MAX_AT,data,numRepe,fitness_pop,dicc_fitness)
    
    for i in range(1,len(pop)):
        c = pop[i]
        f = fitness(pop[i],NUM_MAX_AT,data,numRepe,fitness_pop,dicc_fitness)
        
        if(f > bestFitness):
            bestC = c
            bestFitness = f
    print(dicc_fitness[str(bestC)])
    print(bestC)
    return bestC

def decode(c,data):
    return selectAtributes(c,data)        
    
def runAlgorithm(probMutate,probReproduce,INITIAL_P,NUM_GENERATIONS,NUM_MAX_AT,data,bestChromosomeRepetitions,numRepe):
    umbral_var=0.001
    columns = data.shape[1]
    dicc_fitness = dict()
    fitness_pop = []
    pop = initialPop(columns,INITIAL_P)
    print("Pop 1")
    print(pop)
    print("Pop size: ",len(pop))
    bestC = pop[0]
    counter = 0
    for i in range(0,NUM_GENERATIONS):
        if counter==bestChromosomeRepetitions:
            print('La población se ha estancado porque el mejor cromosoma es siempre el mismo')
            break
        else:
            pop = nextGeneration(pop,[probMutate,1.0-probMutate],[probReproduce,1.0-probReproduce]
                                 ,INITIAL_P,NUM_MAX_AT,data,numRepe,fitness_pop,dicc_fitness)
            print("Pop ",i+2)    
            print(pop)
            print("Pop size: ",len(pop))
            bestSol = bestSolution(pop,NUM_MAX_AT,data,numRepe,fitness_pop,dicc_fitness)
            if bestC == bestSol:
                counter+=1
            else:
                counter = 1
                bestC = bestSol
            if var(fitness_pop) <umbral_var:
                print('La población se ha estancado por la varianza')
                break
        fitness_pop.clear()
    bs = bestSolution(pop,NUM_MAX_AT,data,numRepe,fitness_pop,dicc_fitness)        
    f = fitness(bs,NUM_MAX_AT,data,numRepe,fitness_pop,dicc_fitness)
    d = decode(bs,data)
    print("Función de evaluacion de la mejor solución: ",dicc_fitness[str(bs)])
    print(max(dicc_fitness.values()))
    print(d)
    dicc_fitness.clear()

In [13]:
csv_data = pandas.read_csv('conjuntos_de_datos/BreastCancerDataset.csv', header=0)
csv_data2 = pandas.read_csv('conjuntos_de_datos/parkinson.csv', header=0)
csv_data3 = pandas.read_csv('conjuntos_de_datos/pcos.csv', header=0)
csv_data4 = pandas.read_csv('conjuntos_de_datos/titanic.csv', header=0)
INITIAL_P =  100 # Must be an even number
NUM_GENERATIONS = 30 # Number of chromosomes to generate
NUM_MAX_AT = 7 # Maximum number of atributes to aim for
dicc_fitness = dict() # Empty dict, don't change it
bestChromosomeRepetitions = 5 # Maximum number of the same best chromosome in consecutive populations
probMutate=0.6 # Mutation probability
probReproduce=0.9 # Reproduction probability
umbral_var = 0.001 # Variance treshold
fitness_pop = []  # Empty list, don't change it
numRepe = 5 # Number of cross validation folds
runAlgorithm(probMutate,probReproduce,INITIAL_P,NUM_GENERATIONS,NUM_MAX_AT,csv_data3,bestChromosomeRepetitions,numRepe)

Pop 1
[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1], [1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0], [1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0], [0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1], [1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1], [0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1], [1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1], [1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1], [0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1

-881.3557675322381
[1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0]
-1425.5551139374668
[1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0]
-656.6699788023318
[1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1]
-1420.2585673909202
[1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1]
-1143.3368662780426
[0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0]
-1139.616454689984
[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1]
-1143.5636371665782
[0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0]
-1137.5989224518637
[1, 0, 0, 1, 0, 0

-1420.2208090443385
[0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0]
-173.76921038685745
[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]
-1144.6056350468116
[0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0]
-173.11053700759584
[0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0]
-1420.2208090443385
[0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0]
-1143.3368662780426
[0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0]
-173.11053700759584
[0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0]
-1143.3368662780426
[0, 0, 0, 0, 0

-55.739268680445164
[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0]
-307.8422540187246
[0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]
-305.73198198198196
[1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]
-469.70058293587704
[0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0]
-461.8126214449744
[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0]
-55.739268680445164
[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0]
-310.96361066949305
[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0]
-76.89697049991167
[1, 0, 0, 0, 1, 0

52.02261084614025
[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
8.343711358417238
[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]
52.02261084614025
[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
-2.032105635046804
[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]
41.584084084084076
[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]
52.02261084614025
[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
8.343711358417238
[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]
-68.67183359830419
[0, 1, 0, 0, 0, 0, 0, 1, 0,

...

86.94422363540009
[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]
84.6416269210387
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0]
85.83686627804276
[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]
85.18592121533297
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0]
85.18592121533297
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0]
84.06222398869458
[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]
84.77300830242007
[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]
86.64635223458752
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 

87.21824765942412
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0]
85.09472708002119
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]
85.76399929341105
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]
86.52755696873345
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]
85.27247836071365
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0]
87.21824765942412
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0]
86.48869457692989
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
86.95680975092739
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0,

86.95680975092739
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]
85.70923865041513
[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]
86.09079667903195
[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]
86.95680975092739
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0]
85.32856385797564
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]
86.22593181416711
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0]
85.70923865041513
[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]
86.95680975092739
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0,

87.64750044161808
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0]
86.38866807984455
[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]
85.65028263557674
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]
85.79667903197314
[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]
85.65028263557674
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]
85.51890125419538
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]
87.23458752870516
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0]
86.22593181416711
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0,

87.51611906023669
[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0]
87.51611906023669
[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0]
Función de evaluacion de la mejor solución:  87.51611906023669
87.64750044161808
     Pulse rate(bpm)   Cycle(R/I)  Weight gain(Y/N)  hair growth(Y/N)  \
0                  78           2                 0                 0   
1                  74           2                 0                 0   
2                  72           2                 0                 0   
3                  72           2                 0                 0   
4                  72           2                 0                 0   
..                ...         ...               ...               ...   
536                72           2                 0                 0   
537                72           2                 1            